In [1]:
# using GLMakie, GraphMakie
# using GraphMakie.NetworkLayout
using Graphs, GraphIO

using CSV, DataFrames

using NetworkLayout
using GeometryBasics

using GMT

using FileIO

include("./src/cubes.jl")
include("./src/network.jl")

add_properties (generic function with 1 method)

In [2]:
# Read data
path = "./data/"
region = "romania"
filepath = path * region * ".csv"

mkpath("./maps/$region")

df = CSV.read(filepath, DataFrame);

side = 5
minimum_magnitude = 4

df_filtered = df[df.Magnitude .> minimum_magnitude,:] 

# Split into cubes
df_filtered, df_filtered_cubes = region_cube_split(df_filtered,side=side,energyRelease=true);

# Create network
MG = create_network(df_filtered, df_filtered_cubes)
connectivity = degree(MG);

# edgelist_array = Matrix(edgelist);
edgelist = collect(edges(MG)) |> DataFrame;

In [12]:
# Get region's coordinates
min_lon = minimum(df_filtered_cubes.cubeLongitude)
max_lon = maximum(df_filtered_cubes.cubeLongitude)
min_lat = minimum(df_filtered_cubes.cubeLatitude)
max_lat = maximum(df_filtered_cubes.cubeLatitude);
min_dep = minimum(df_filtered_cubes.cubeDepth);
max_dep = maximum(df_filtered_cubes.cubeDepth);

# Create the map coordinates
map_coords = [min_lon,max_lon,min_lat,max_lat]
map_coords_depth = [min_lon,max_lon,min_lat,max_lat,-max_dep,-min_dep]

# Colormap for the region topography
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
# Relief map of the region
relief_map = grdcut("@earth_relief_30s", region=map_coords);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 21.0817 to 21.075
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 29.5723 to 29.575
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 44.9225 to 44.9166666667
grdcut [WARNING]: (n - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: n reset from 47.9365 to 47.9416666667


In [4]:
graph_path = "./networks/$region/side_$(side)km/$(region)_side_$(side)km_minmag_$(minimum_magnitude).txt"

"./networks/romania/side_5km/romania_side_5km_minmag_4.txt"

In [28]:
# control marker size based on degree
marker_size = connectivity ./10

# control marker color by connectivity
C_markers = makecpt(cmap=:seis, range=(minimum(connectivity),maximum(connectivity)), inverse=true);
zcolor_control = connectivity;

perspective = (135,20)

(135, 20)

In [29]:
# Basemap to define the axes
basemap(limits=map_coords_depth, proj=:merc, zsize=6, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", view=perspective)

# Edges, plotted manually
for i in range(1,nrow(edgelist))
    line_coords = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                    lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                    deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

    plot3d!(line_coords.lons, line_coords.lats, -line_coords.deps, JZ="6c", proj=:merc, pen=(:thinner,:black), alpha=50, view=perspective)
end

# Nodes
scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
limits=map_coords_depth,frame="SEnwZ1+b xafg yafg zaf",proj=:merc, marker=:cube,markersize=0.1, #markersize=marker_size,
cmap=C_markers, zcolor=connectivity, 
alpha=50, view=perspective)

# Colorbar
colorbar!(limits=map_coords, pos=(paper=(16,2.0), size=(8,0.5)), shade=0.4, xaxis=(annot=2,), frame=(xlabel="Degree",),par=(MAP_LABEL_OFFSET=0.6,),view=(180,90))

# Relief map
grdview!(relief_map, proj=:merc, surftype=(image=1000,), 
cmap=C_map, zsize=0.5, alpha=10 ,yshift=5.6, view=perspective,
savefig="./networks/$region/side_$(side)km/$(region)_side_$(side)km_minmag_$(minimum_magnitude).png", 
show=true)

In [24]:
using DelimitedFiles

## Motifs plot polygons

In [25]:
motif="Triangle"
network_target_path ="./networks/$region/side_$(side)km/"
motif_filename = "motif$(motif)_$(region)_side_$(side)km_minmag_$(minimum_magnitude).csv"

"motifTriangle_romania_side_5km_minmag_4.csv"

In [26]:
motifs = readdlm(network_target_path * motif_filename, ',', Int64);

In [42]:
# Basemap to define the axes
basemap(limits=map_coords_depth, proj=:merc, zsize=6, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", view=(135,20))

# Edges, plotted manually
for i in range(1,nrow(edgelist))
    line = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                    lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                    deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

    plot3d!(line.lons,line.lats,-line.deps, JZ="6c", proj=:merc, pen=(:thinner,:black), alpha=50, view=(135,20))
end

# Nodes
scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
limits=map_coords_depth, frame="SEnwZ1+b xafg yafg zaf",proj=:merc, marker=:cube,markersize=0.1, #markersize=marker_size,
cmap=C_markers, zcolor=connectivity, 
alpha=50, view=(135,20))

# Motifs
for i in range(1,size(motifs,1))
    motif = DataFrame(lats = [df_filtered_cubes.cubeLatitude[motifs[i,1]], df_filtered_cubes.cubeLatitude[motifs[i,2]], df_filtered_cubes.cubeLatitude[motifs[i,3]]],
                    lons =[df_filtered_cubes.cubeLongitude[motifs[i,1]], df_filtered_cubes.cubeLongitude[motifs[i,2]], df_filtered_cubes.cubeLongitude[motifs[i,3]]],
                    deps= [df_filtered_cubes.cubeDepth[motifs[i,1]], df_filtered_cubes.cubeDepth[motifs[i,2]], df_filtered_cubes.cubeDepth[motifs[i,3]]])
    plot3d!(motif.lons,motif.lats,-motif.deps, JZ="6c", proj=:merc, L=true, G=:red, alpha=50, view=(135,20))
end

# Colorbar
colorbar!(pos=(outside=:BC, offset=(0,1.5)), shade=0.4, JZ="6c", xaxis=(annot=:auto,), frame=(xlabel="Degree",),par=(MAP_LABEL_OFFSET=0.8,))

# Relief map
grdview!(relief_map, proj=:merc, surftype=(image=2000,), 
cmap=C_map, zsize=0.05, alpha=10 ,yshift=5.6, view=(135,20),
savefig="./networks/$region/side_$(side)km/$(region)_side_$(side)km_minmag_$(minimum_magnitude).png", 
show=true)